# Fair Trading Entity Analysis
This notebook demonstrates:
1. A 'single view' function aggregating all known data for a regulated entity.
2. Building a relationship graph (companies and directors).
3. Finding related entities via shared directors.
4. Detecting potential phoenix patterns based on director movements.

Example DataFrames are provided; replace with real data for production use.

In [1]:
# Imports
import pandas as pd
import networkx as nx


## Identifier Normalization

In [2]:
# Normalize ACN/ABN by removing spaces (example). Adjust as needed for real formats.
def normalize_id(val):
    if isinstance(val, str):
        return val.replace(' ', '')
    return val


## Example DataFrames

In [3]:
# Example ASIC directors/officers data
asic_directors_example = pd.DataFrame([
    {'ACN': '123456789', 'CompanyName': 'ABC Pty Ltd', 'DirectorName': 'John Smith', 'AppointmentDate': '2020-05-15', 'CessationDate': None},
    {'ACN': '987654321', 'CompanyName': 'XYZ Holdings Pty Ltd', 'DirectorName': 'Jane Doe', 'AppointmentDate': '2018-03-10', 'CessationDate': '2023-11-30'}
])
# Example ABR data
abr_example = pd.DataFrame([
    {'ABN': '12 345 678 901', 'LegalName': 'Alpha Innovations Pty Ltd', 'BusinessNames': ['Alpha Innovations'], 'TradingNames': ['Alpha Tech', 'Alpha Solutions'], 'State': 'NSW', 'Postcode': '2000', 'ACN': '123456789', 'GSTStatus': 'Active', 'RegistrationDate': '2019-07-01'},
    {'ABN': '98 765 432 109', 'LegalName': 'Beta Services Pty Ltd', 'BusinessNames': ['Beta Services'], 'TradingNames': ['Beta Consulting'], 'State': 'NSW', 'Postcode': '2140', 'ACN': '987654321', 'GSTStatus': 'Active', 'RegistrationDate': '2021-02-15'}
])
# Example complaints
complaints_example = pd.DataFrame([
    {'ComplaintID': 'C202401001', 'BusinessName': 'ABC Pty Ltd', 'ABN': '12 345 678 901', 'DateReceived': '2024-06-01', 'IssueCategory': 'Product quality', 'Status': 'Resolved'},
    {'ComplaintID': 'C202402015', 'BusinessName': 'XYZ Holdings Pty Ltd', 'ABN': '98 765 432 109', 'DateReceived': '2024-05-20', 'IssueCategory': 'Misleading advertising', 'Status': 'Under investigation'}
])
# Example inspections
inspections_example = pd.DataFrame([
    {'InspectionID': 'I20230101', 'BusinessName': 'ABC Pty Ltd', 'ABN': '12 345 678 901', 'InspectionDate': '2023-10-15', 'Location': 'Sydney', 'Outcome': 'Compliant', 'InspectorName': 'Alice Brown', 'Notes': 'No issues found'},
    {'InspectionID': 'I20230202', 'BusinessName': 'Beta Services Pty Ltd', 'ABN': '98 765 432 109', 'InspectionDate': '2024-03-22', 'Location': 'Parramatta', 'Outcome': 'Non-compliant', 'InspectorName': 'Bob Green', 'Notes': 'Labeling issues; follow-up required'}
])
# Example investigations
investigations_example = pd.DataFrame([
    {'InvestigationID': 'INV2024001', 'BusinessName': 'XYZ Holdings Pty Ltd', 'ABN': '98 765 432 109', 'StartDate': '2024-05-25', 'EndDate': None, 'Outcome': 'Ongoing', 'RelatedEntities': ['Alpha Innovations Pty Ltd']},
    {'InvestigationID': 'INV2023005', 'BusinessName': 'Gamma Trading Pty Ltd', 'ABN': '11 222 333 444', 'StartDate': '2023-01-10', 'EndDate': '2023-12-05', 'Outcome': 'Enforcement action taken', 'RelatedEntities': []}
])
# Normalize IDs in example DataFrames
asic_directors_example['ACN'] = asic_directors_example['ACN'].apply(normalize_id)
abr_example['ACN'] = abr_example['ACN'].apply(normalize_id)
abr_example['ABN'] = abr_example['ABN'].apply(normalize_id)
complaints_example['ABN'] = complaints_example['ABN'].apply(normalize_id)
inspections_example['ABN'] = inspections_example['ABN'].apply(normalize_id)
investigations_example['ABN'] = investigations_example['ABN'].apply(normalize_id)
# Display examples (optional)
print('ASIC directors example:'); display(asic_directors_example)
print('ABR example:'); display(abr_example)


ASIC directors example:


,ACN,CompanyName,DirectorName,AppointmentDate,CessationDate
0,123456789,ABC Pty Ltd,John Smith,2020-05-15,None
1,987654321,XYZ Holdings Pty Ltd,Jane Doe,2018-03-10,2023-11-30


ABR example:


,ABN,LegalName,BusinessNames,TradingNames,State,Postcode,ACN,GSTStatus,RegistrationDate
0,12345678901,Alpha Innovations Pty Ltd,[Alpha Innovations],"[Alpha Tech, Alpha Solutions]",NSW,2000,123456789,Active,2019-07-01
1,98765432109,Beta Services Pty Ltd,[Beta Services],[Beta Consulting],NSW,2140,987654321,Active,2021-02-15


## Single View Function

In [12]:
def get_single_view(acn=None, abn=None):
    """Return aggregated info for an entity by ACN or ABN."""
    result = {}
    # If ACN provided: fetch ASIC and ABR info
    if acn:
        acn_norm = normalize_id(acn)
        df_dir = asic_directors_example[asic_directors_example['ACN'] == acn_norm]
        if not df_dir.empty:
            result['CompanyName_ASIC'] = df_dir.iloc[0]['CompanyName']
        df_abr = abr_example[abr_example['ACN'] == acn_norm]
        if not df_abr.empty:
            abr_row = df_abr.iloc[0]
            result.update({
                'LegalName_ABR': abr_row['LegalName'],
                'BusinessNames': abr_row['BusinessNames'],
                'TradingNames': abr_row['TradingNames'],
                'ABN': abr_row['ABN'],
                'State': abr_row['State'],
                'Postcode': abr_row['Postcode'],
                'GSTStatus': abr_row['GSTStatus'],
                'RegistrationDate': abr_row['RegistrationDate']
            })
        else:
            result['ABR_record'] = 'Not found'
    # If ABN provided: fetch ABR, and related data
    if abn:
        abn_norm = normalize_id(abn)
        df_abr2 = abr_example[abr_example['ABN'] == abn_norm]
        if not df_abr2.empty:
            abr_row2 = df_abr2.iloc[0]
            result.setdefault('LegalName_ABR', abr_row2['LegalName'])
            result.setdefault('BusinessNames', abr_row2['BusinessNames'])
            result.setdefault('TradingNames', abr_row2['TradingNames'])
            result.setdefault('ACN', abr_row2['ACN'])
            result.setdefault('State', abr_row2['State'])
            result.setdefault('Postcode', abr_row2['Postcode'])
            result.setdefault('GSTStatus', abr_row2['GSTStatus'])
            result.setdefault('RegistrationDate', abr_row2['RegistrationDate'])
        else:
            result['ABR_record'] = 'Not found'
        # Complaints
        df_com = complaints_example[complaints_example['ABN'] == abn_norm]
        result['Complaints'] = df_com.to_dict(orient='records') if not df_com.empty else []
        # Inspections
        df_ins = inspections_example[inspections_example['ABN'] == abn_norm]
        result['Inspections'] = df_ins.to_dict(orient='records') if not df_ins.empty else []
        # Investigations
        df_inv = investigations_example[investigations_example['ABN'] == abn_norm]
        result['Investigations'] = df_inv.to_dict(orient='records') if not df_inv.empty else []
    # Directors if ACN known
    if 'ACN' in result:
        df_dir2 = asic_directors_example[asic_directors_example['ACN'] == normalize_id(result['ACN'])]
        result['Directors'] = df_dir2.to_dict(orient='records') if not df_dir2.empty else []
    return result


### Demonstrate Single View

In [5]:
# Single view for example entities
sv_abc = get_single_view(acn='123456789')
sv_xyz = get_single_view(abn='98 765 432 109')
print('Single view ABC Pty Ltd:'); sv_abc
print('Single view XYZ Holdings Pty Ltd:'); sv_xyz

Single view ABC Pty Ltd:
Single view XYZ Holdings Pty Ltd:


{'LegalName_ABR': 'Beta Services Pty Ltd',
 'BusinessNames': ['Beta Services'],
 'TradingNames': ['Beta Consulting'],
 'ACN': '987654321',
 'State': 'NSW',
 'Postcode': '2140',
 'GSTStatus': 'Active',
 'RegistrationDate': '2021-02-15',
 'Complaints': [{'ComplaintID': 'C202402015',
   'BusinessName': 'XYZ Holdings Pty Ltd',
   'ABN': '98765432109',
   'DateReceived': '2024-05-20',
   'IssueCategory': 'Misleading advertising',
   'Status': 'Under investigation'}],
 'Inspections': [{'InspectionID': 'I20230202',
   'BusinessName': 'Beta Services Pty Ltd',
   'ABN': '98765432109',
   'InspectionDate': '2024-03-22',
   'Location': 'Parramatta',
   'Outcome': 'Non-compliant',
   'InspectorName': 'Bob Green',
   'Notes': 'Labeling issues; follow-up required'}],
 'Investigations': [{'InvestigationID': 'INV2024001',
   'BusinessName': 'XYZ Holdings Pty Ltd',
   'ABN': '98765432109',
   'StartDate': '2024-05-25',
   'EndDate': None,
   'Outcome': 'Ongoing',
   'RelatedEntities': ['Alpha Innovatio

## Relationship Graph

In [13]:
def build_relationship_graph():
    G = nx.Graph()
    for _, row in asic_directors_example.iterrows():
        comp_node = f"ACN:{normalize_id(row['ACN'])}"
        person_node = f"Person:{row['DirectorName']}"
        G.add_node(comp_node, type='company')
        G.add_node(person_node, type='person')
        G.add_edge(comp_node, person_node,
                   relation='director',
                   appointment=row['AppointmentDate'],
                   cessation=row['CessationDate'])
    return G


### Finding Related Entities

In [7]:
def find_related_entities(graph, acn):
    """Return ACNs related via shared directors."""
    node = f"ACN:{normalize_id(acn)}"
    related = set()
    if not graph.has_node(node):
        return related
    for person in graph.neighbors(node):
        for nbr in graph.neighbors(person):
            if nbr.startswith('ACN:') and nbr != node:
                related.add(nbr.replace('ACN:', ''))
    return related


### Demonstrate Related Entities

In [8]:
G = build_relationship_graph()
related_abc = find_related_entities(G, acn='123456789')
print('Related ACNs for ABC Pty Ltd:', related_abc)


Related ACNs for ABC Pty Ltd: set()


## Phoenix Pattern Detection

In [14]:
def detect_phoenix_candidates(asic_df, window_days=180):
    """Identify director movements suggesting phoenix behavior."""
    # Parse dates
    df = asic_df.copy()
    df['AppointmentDate'] = pd.to_datetime(df['AppointmentDate'])
    df['CessationDate'] = pd.to_datetime(df['CessationDate'], errors='coerce')
    candidates = []
    for person, group in df.groupby('DirectorName'):
        group_sorted = group.sort_values('AppointmentDate')
        for i, row_a in group_sorted.iterrows():
            if pd.isna(row_a['CessationDate']):
                continue
            cease_date = row_a['CessationDate']
            later = group_sorted[group_sorted['AppointmentDate'] > cease_date]
            for j, row_b in later.iterrows():
                if (row_b['AppointmentDate'] - cease_date).days <= window_days:
                    candidates.append({
                        'Director': person,
                        'OldACN': row_a['ACN'],
                        'NewACN': row_b['ACN'],
                        'CessationDate': cease_date.date(),
                        'NewAppointmentDate': row_b['AppointmentDate'].date()
                    })
    return pd.DataFrame(candidates)


### Demonstrate Phoenix Detection

In [10]:
# Using example asic_directors_example
phoenix_df = detect_phoenix_candidates(asic_directors_example)
print('Phoenix candidates (example):'); phoenix_df

Phoenix candidates (example):


""


## Next Steps
1. Replace example DataFrames with real datasets loaded from APIs or databases.
2. Enhance single view: include summary counts, last dates, risk scoring.
3. Extend graph: add shareholding edges, investigation links, addresses.
4. Scale graph: consider graph database (e.g., Neo4j) for large volumes.
5. Automate: schedule regular updates and alerts for new phoenix patterns.
